In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,596 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://archive.ubuntu.com/

In [ ]:
import pandas as pd
# import pandasql as ps
# # import pixiedust
# import sys
# sys.path.append('..')
# import util
# import etl
# import pyarrow.parquet as pq
# import pyarrow as pa
# !pip install duckdb
# import duckdb
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *
# spark = SparkSession.builder.getOrCreate()

# util.usedatabase(spark, "real_world_data_jun_2022")

# pixiedust.enableJobMonitor()

# con = duckdb.connect()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cohort_new = spark.read.parquet('/content/drive/MyDrive/UCO/projects/epilepsy/allcohort2024')

In [ ]:
cohort_demo = spark.read.parquet('/content/drive/MyDrive/UCO/projects/epilepsy/allcohort_new_demo')

In [ ]:
cohort_demo.count()
# 1701685

1701685

In [ ]:
cohort_como = spark.read.parquet('/content/drive/MyDrive/UCO/projects/epilepsy/allcohort_new_como')

In [ ]:
cohort_como.show(5)

+--------------------+--------------------+-------------+--------------------+
|            personid|         encounterid|comorbidityid|       effectivedate|
+--------------------+--------------------+-------------+--------------------+
|00169bfe-c751-491...|fd5f3f12-37ca-418...|       Z34.82|2022-03-21T12:44:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        Z33.1|2021-11-28T20:07:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        U07.1|2021-11-28T20:13:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        Z45.2|2022-01-04T15:02:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        E04.9|2022-01-02T17:00:...|
+--------------------+--------------------+-------------+--------------------+
only showing top 5 rows



In [ ]:
cohort_como_reformat= cohort_como.withColumn('comorbidityid',expr("regexp_replace(comorbidityid, '([0-9a-zA-Z]+\\.[0-9a-zA-Z]).*', '$1')").cast("string"))

In [ ]:
cohort_como_reformat.show(5)

+--------------------+--------------------+-------------+--------------------+
|            personid|         encounterid|comorbidityid|       effectivedate|
+--------------------+--------------------+-------------+--------------------+
|00169bfe-c751-491...|fd5f3f12-37ca-418...|        Z34.8|2022-03-21T12:44:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        Z33.1|2021-11-28T20:07:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        U07.1|2021-11-28T20:13:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        Z45.2|2022-01-04T15:02:...|
|00169bfe-c751-491...|f42c4077-f0cf-469...|        E04.9|2022-01-02T17:00:...|
+--------------------+--------------------+-------------+--------------------+
only showing top 5 rows



In [ ]:
def get_top_comorbidity(spark, como, topn):
    top_como = como.groupBy('comorbidityid').count().orderBy(col('count').desc())\
        .select(collect_list('comorbidityid')).collect()[0]\
            .__getitem__('collect_list(comorbidityid)')[0:topn]

#     top_como_coded = ["n"+x.replace('.','_') for x in top_como]
#     out = {top_como[i]:top_como_coded[i] for i in range(len(top_como))} # store in a dictionary
    print(top_como)
    return top_como

In [ ]:
top_comorbidity = get_top_comorbidity(spark, cohort_como_reformat, topn=20)

['I10', 'Z79.8', 'Z00.1', 'Z23', 'M25.5', 'E78.5', 'E11.9', 'W19.X', 'J06.9', 'I25.1', 'E03.9', 'K21.9', 'Y92.0', 'Z79.0', 'F41.9', '401.9', 'R51', 'R05', 'I48.9', 'M79.6']


In [ ]:
cohort_como_length = cohort_como_reformat.join(cohort_new.select('personid','date'),'personid').withColumn('len_como',months_between(col('date'),col('effectivedate'))/12)

In [ ]:
# prompt: make the len_como integer

cohort_como_length = cohort_como_length.withColumn("len_como", col("len_como").cast("integer"))


In [ ]:
cohort_como_length_filter = cohort_como_length.filter(col('comorbidityid').isin(top_comorbidity))

In [ ]:
cohort_como_length_filter.show(5)

+--------------------+--------------------+-------------+--------------------+--------------------+--------+
|            personid|         encounterid|comorbidityid|       effectivedate|                date|len_como|
+--------------------+--------------------+-------------+--------------------+--------------------+--------+
|00010e6e-593a-4e1...|c8a0da6a-aadd-47d...|        M79.6|2016-01-29T04:15:...|2020-03-17T03:09:...|       4|
|00010e6e-593a-4e1...|87247b0f-a8f0-45d...|        Z79.8|2020-02-28T19:30:...|2020-03-17T03:09:...|       0|
|00010e6e-593a-4e1...|98771808-b475-443...|        K21.9|2019-09-02T03:22:...|2020-03-17T03:09:...|       0|
|00010e6e-593a-4e1...|7780363b-9753-449...|        K21.9|2019-03-09T03:33:...|2020-03-17T03:09:...|       1|
|00010e6e-593a-4e1...|069c9d67-7cc9-440...|        Y92.0|2016-12-03T09:44:...|2020-03-17T03:09:...|       3|
+--------------------+--------------------+-------------+--------------------+--------------------+--------+
only showing top 5 

In [ ]:
# prompt: keep personid, comorbidityid and len_como in cohort_como_length_filter and remove duplicates

cohort_como_length_filter_selected = cohort_como_length_filter.select('personid', 'comorbidityid', 'len_como').dropDuplicates()
# cohort_como_length_filter_selected.show(5)


In [ ]:
cohort_como_length_filter.count()

6661834

In [ ]:
cohort_demo.show(5)

▸,:,


<IPython.core.display.Javascript object>

+--------------------+--------------------+------+--------------------+-----+
|            personid|           birthdate|gender|                date| race|
+--------------------+--------------------+------+--------------------+-----+
|00000559-6f93-4e9...|[[1970-02-19], 19...|Female|2017-02-02T19:29:...|White|
|00003900-2614-4da...|[[2009-09-05], 20...|  Male|2015-09-27T04:00:...|White|
|0001df3b-fec4-4e9...|[[1984-04-01], 19...|Female| 2016-03-01T00:00:00|White|
|000538ff-d137-48e...|[[1974-10-16], 19...|  Male|2017-06-22T05:00:...| null|
|001517b6-1588-409...|[[2015-02-10], 20...|  Male| 2020-07-27T00:00:00|White|
+--------------------+--------------------+------+--------------------+-----+
only showing top 5 rows



<IPython.core.display.Javascript object>

In [ ]:
cohort_demo_processed = cohort_demo.withColumn('age_at_diagnosis',months_between(col('date'),col('birthdate.value'))/12)
cohort_demo_processed = cohort_demo_processed.withColumn('age_at_diagnosis',col('age_at_diagnosis').cast("integer"))
cohort_demo_processed = cohort_demo_processed.drop('birthdate')
# cohort_demo_processed.show(5)

In [ ]:
# Preprocess gender column
df = cohort_demo_processed
df = df.withColumn("gender",
                   when(lower(col("gender")).isin("male"), 1)
                   .when(lower(col("gender")).isin("female"), 2)
                   .when(col("gender").isNull(), 0)
                   .otherwise(3))

# Preprocess race column
df = df.withColumn("race",
                   when(lower(col("race")).like("%white%") | lower(col("race")).like("%caucasian%"), 1)
                   .when(lower(col("race")).like("%black%") | lower(col("race")).like("%african%"), 2)
                   .when(lower(col("race")).like("%hispanic%"), 3)
                   .when(lower(col("race")).like("%asian%") | lower(col("race")).like("%chinese%") | lower(col("race")).like("%korean%") | lower(col("race")).like("%japanese%"), 4)
                   .when(lower(col("race")).like("%indian american%") | lower(col("race")).like("%native american%"), 5)
                   .when(col("race").isNull(), 0)
                   .otherwise(6))
df.show()
cohort_demo_processed = df

+--------------------+------+--------------------+----+----------------+
|            personid|gender|                date|race|age_at_diagnosis|
+--------------------+------+--------------------+----+----------------+
|00000559-6f93-4e9...|     2|2017-02-02T19:29:...|   1|              46|
|00003900-2614-4da...|     1|2015-09-27T04:00:...|   1|               6|
|0001df3b-fec4-4e9...|     2| 2016-03-01T00:00:00|   1|              31|
|000538ff-d137-48e...|     1|2017-06-22T05:00:...|   0|              42|
|001517b6-1588-409...|     1| 2020-07-27T00:00:00|   1|               5|
|00179a60-4542-4d4...|     1|2020-03-27T11:35:...|   1|              81|
|001a2011-9523-415...|     1|2022-01-08T12:11:...|   1|              26|
|0021fe8a-5e63-441...|     1|2019-11-28T18:00:...|   1|               2|
|0021ff9b-1406-472...|     1|2018-01-11T08:00:...|   1|              26|
|0029b094-5715-477...|     2|                    |   2|            NULL|
|00384f92-4a1a-4e7...|     1|2020-04-14T18:26:...| 

In [ ]:
person_vertice = cohort_demo_processed
person_vertice = person_vertice.withColumn("id", col('personid')).drop('personid','date')
diagnosis_vertice = cohort_como_length_filter.select("comorbidityid").distinct()
diagnosis_vertice = diagnosis_vertice.withColumn("id", col('comorbidityid')).drop(col('comorbidityid'))

In [ ]:
person_vertice.select('id').count()

1701685

In [ ]:
person_vertice.select('id').distinct().count()

1701685

In [ ]:
# print(person_vertice.columns)
# print(diagnosis_vertice.columns)

['gender', 'race', 'age_at_diagnosis', 'id']
['id']


In [ ]:
# def union_columns(df1, df2):
#     diff1 = [c for c in df2.columns if c not in df1.columns]
#     diff2 = [c for c in df1.columns if c not in df2.columns]
#     df = df1.select('*', *[lit(None).alias(c) for c in diff1]) \
#         .unionByName(df2.select('*', *[lit(None).alias(c) for c in diff2]))
#     return df

In [ ]:
# All_vertices = union_columns(person_vertice, diagnosis_vertice)

In [ ]:
personid_diagnosis_edges = cohort_new.join(cohort_como_length_filter,'personid').select("personid", "comorbidityid",'len_como')
# personid_diagnosis_edges = personid_diagnosis_edges.withColumn("cormorbidity_year", col('comorbidity_year'))
personid_diagnosis_edges = personid_diagnosis_edges.withColumn("src", col('personid'))
personid_diagnosis_edges = personid_diagnosis_edges.withColumn("dst", col('comorbidityid'))

diagnosis_personid_edges = personid_diagnosis_edges.withColumn("src", col('comorbidityid'))
diagnosis_personid_edges = personid_diagnosis_edges.withColumn("dst", col('personid'))

personid_diagnosis_edges = personid_diagnosis_edges.drop(col('personid')).drop(col('comorbidityid'))
diagnosis_personid_edges = diagnosis_personid_edges.drop(col('personid')).drop(col('comorbidityid'))

In [ ]:
# all_edges = personid_diagnosis_edges.unionAll(diagnosis_personid_edges)
all_edges = personid_diagnosis_edges

In [ ]:
all_edges.show(5)

+--------+--------------------+-----+
|len_como|                 src|  dst|
+--------+--------------------+-----+
|       4|00010e6e-593a-4e1...|M79.6|
|       0|00010e6e-593a-4e1...|Z79.8|
|       0|00010e6e-593a-4e1...|K21.9|
|       1|00010e6e-593a-4e1...|K21.9|
|       3|00010e6e-593a-4e1...|Y92.0|
+--------+--------------------+-----+
only showing top 5 rows



In [ ]:
# Create input features tensor
# import torch

# # Mapping person and comorbidity IDs to indices
# person_id_to_idx = {id_: idx for idx, id_ in enumerate(person_vertice.toPandas()['id'])}
# comorbidity_id_to_idx = {id_: idx + len(person_id_to_idx) for idx, id_ in enumerate(diagnosis_vertice.toPandas()['id'])}
# id_to_idx = {**person_id_to_idx, **comorbidity_id_to_idx}

In [ ]:
# revised to put comorbidity indexes at the beginning
import torch

# Mapping person and comorbidity IDs to indices
comorbidity_id_to_idx = {id_: idx  for idx, id_ in enumerate(diagnosis_vertice.toPandas()['id'])}
person_id_to_idx = {id_: idx + len(comorbidity_id_to_idx) for idx, id_ in enumerate(person_vertice.toPandas()['id'])}

id_to_idx = {**comorbidity_id_to_idx, **person_id_to_idx}

In [ ]:
# prompt: convert id of person_vertice using id_to_idx

# Assuming 'person_vertice' is a PySpark DataFrame
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# Create a UDF to map person IDs to indices
map_person_id_udf = udf(lambda id_: id_to_idx.get(id_), IntegerType())

# Apply the UDF to the 'id' column of 'person_vertice'
person_vertice_indexed = person_vertice.withColumn("id_index", map_person_id_udf(col("id")))

# Show the updated DataFrame
# person_vertice_indexed.show()


In [ ]:
# prompt: convert id of diagnosis_vertice using id_to_idx

# Create a UDF to map comorbidity IDs to indices
map_comorbidity_id_udf = udf(lambda id_: id_to_idx.get(id_), IntegerType())

# Apply the UDF to the 'id' column of 'diagnosis_vertice'
diagnosis_vertice_indexed = diagnosis_vertice.withColumn("id_index", map_comorbidity_id_udf(col("id")))

# Show the updated DataFrame
# diagnosis_vertice_indexed.show()


In [ ]:
# prompt: convert src and dst of all_edges using id_to_idx

# Create a UDF to map IDs to indices for 'src' column
map_src_id_udf = udf(lambda id_: id_to_idx.get(id_), IntegerType())

# Apply the UDF to the 'src' column of 'all_edges'
all_edges_indexed = all_edges.withColumn("src_index", map_src_id_udf(col("src")))

# Create a UDF to map IDs to indices for 'dst' column
map_dst_id_udf = udf(lambda id_: id_to_idx.get(id_), IntegerType())

# Apply the UDF to the 'dst' column of 'all_edges_indexed'
all_edges_indexed = all_edges_indexed.withColumn("dst_index", map_dst_id_udf(col("dst")))

# Show the updated DataFrame
# all_edges_indexed.show()


In [ ]:
person_vertice_indexed.show()

+------+----+----------------+--------------------+--------+
|gender|race|age_at_diagnosis|                  id|id_index|
+------+----+----------------+--------------------+--------+
|     2|   1|              46|00000559-6f93-4e9...|      20|
|     1|   1|               6|00003900-2614-4da...|      21|
|     2|   1|              31|0001df3b-fec4-4e9...|      22|
|     1|   0|              42|000538ff-d137-48e...|      23|
|     1|   1|               5|001517b6-1588-409...|      24|
|     1|   1|              81|00179a60-4542-4d4...|      25|
|     1|   1|              26|001a2011-9523-415...|      26|
|     1|   1|               2|0021fe8a-5e63-441...|      27|
|     1|   1|              26|0021ff9b-1406-472...|      28|
|     2|   2|            NULL|0029b094-5715-477...|      29|
|     1|   1|              36|00384f92-4a1a-4e7...|      30|
|     2|   1|              17|003bd84f-f6bd-40a...|      31|
|     2|   1|               1|003eb796-70b0-430...|      32|
|     1|   1|           

In [ ]:
# prompt: convert null in age at diagnosis to 0

person_vertice_indexed = person_vertice_indexed.fillna({'age_at_diagnosis': 0})


In [ ]:
# # prompt: handle null value of x_person

# # Handle null values in x_person
# x_person = torch.nan_to_num(x_person, nan=0)  # Replace NaN with 0.0

# print(x_person)


In [ ]:
# prompt: format person_vertice to x_person, format diagnosis_vertice to x_diagnosis

# Assuming 'person_vertice' is a PySpark DataFrame with columns: 'id_index', 'age_at_diagnosis', 'gender', 'race'
x_person = torch.tensor(person_vertice_indexed.select("id_index", "age_at_diagnosis", "gender", "race")
                         .toPandas().values, dtype=torch.int)

# Assuming 'diagnosis_vertice' is a PySpark DataFrame with column: 'id_index'
x_diagnosis = torch.tensor(diagnosis_vertice_indexed.select("id_index")
                            .toPandas().values, dtype=torch.int)

# print(x_person)
# print(x_diagnosis)


In [ ]:
# prompt: pad x_diagnosis with 0 so that it has the same dimensions except for dimension 0

# Pad x_diagnosis with zeros
padding_size = x_person.shape[1] - x_diagnosis.shape[1]
padding = torch.zeros(x_diagnosis.shape[0], padding_size)
x_diagnosis_padded = torch.cat([x_diagnosis, padding], dim=1).float()

print(x_diagnosis_padded)


tensor([[ 0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.],
        [ 2.,  0.,  0.,  0.],
        [ 3.,  0.,  0.,  0.],
        [ 4.,  0.,  0.,  0.],
        [ 5.,  0.,  0.,  0.],
        [ 6.,  0.,  0.,  0.],
        [ 7.,  0.,  0.,  0.],
        [ 8.,  0.,  0.,  0.],
        [ 9.,  0.,  0.,  0.],
        [10.,  0.,  0.,  0.],
        [11.,  0.,  0.,  0.],
        [12.,  0.,  0.,  0.],
        [13.,  0.,  0.,  0.],
        [14.,  0.,  0.,  0.],
        [15.,  0.,  0.,  0.],
        [16.,  0.,  0.,  0.],
        [17.,  0.,  0.,  0.],
        [18.,  0.,  0.,  0.],
        [19.,  0.,  0.,  0.]])


In [ ]:
# # Merge person and diagnosis features
# x_combined = torch.cat([x_person, x_diagnosis_padded], dim=0)
# print(x_combined)

In [ ]:
# Merge person and diagnosis features, put diagnosis first
x_combined = torch.cat([x_diagnosis_padded, x_person], dim=0)
print(x_combined)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        ...,
        [1.7017e+06, 3.3000e+01, 2.0000e+00, 1.0000e+00],
        [1.7017e+06, 8.0000e+01, 2.0000e+00, 1.0000e+00],
        [1.7017e+06, 0.0000e+00, 3.0000e+00, 1.0000e+00]])


In [ ]:
# prompt: format all_edges to the structure of edge_index_person_to_diagnosis, the edge_attr will be len_como

# Extract source and destination indices and edge attributes
src_indices = all_edges_indexed.select("src_index").rdd.flatMap(lambda x: x).collect()
# dst_indices = all_edges_indexed.select("dst_index").rdd.flatMap(lambda x: x).collect()
# edge_attrs = all_edges_indexed.select("len_como").toPandas().values



In [ ]:
# dst_indices = all_edges_indexed.select("dst_index").toPandas().values
dst_indices = all_edges_indexed.select("dst_index").rdd.flatMap(lambda x: x).collect()

In [ ]:
# edge_index_person_to_diagnosis = torch.tensor([src_indices, dst_indices]).contiguous()
edge_index_person_to_diagnosis = torch.tensor([src_indices, dst_indices]).contiguous()

print(edge_index_person_to_diagnosis)

tensor([[1484397, 1484397, 1484397,  ...,  138541,  138541,  138541],
        [     18,      14,      19,  ...,      16,      11,       6]])


In [ ]:
edge_index_diagnosis_to_person = torch.tensor([dst_indices, src_indices]).contiguous()
# edge_index_diagnosis_to_person = torch.tensor([dst_indices[:,0].tolist(), src_indices]).contiguous()
# print(edge_index_person_to_diagnosis)

In [ ]:
edge_attrs = all_edges_indexed.select("len_como").rdd.map(lambda x: x[0]).collect()
# edge_attrs = all_edges_indexed.select("len_como").toPandas().values[:,0].tolist()

# Create edge attribute tensor
edge_attr = torch.tensor(edge_attrs, dtype=torch.float).unsqueeze(1)  # Unsqueeze to match the expected shape
print(edge_attr)

tensor([[4.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])


In [ ]:
# prompt: transform the cohort_new so that personid is converted to indexes using id_to_idx. EPI is the label. Make sure the label is aligned with id in x_person. Load the label to y and prepare data to be loaded using dataloader

# Create a UDF to map person IDs to indices for labels
map_person_id_for_label_udf = udf(lambda id_: id_to_idx.get(id_), IntegerType())

# Apply the UDF to the 'personid' column of 'cohort_new' and select 'EPI' as label
cohort_new_indexed = cohort_new.withColumn("id_index", map_person_id_for_label_udf(col("personid"))).select('id_index','EPI')

# align with x_person by doing sorting
cohort_new_indexed = cohort_new_indexed.orderBy('id_index')
# display(cohort_new_indexed.show(5))

# Extract labels and convert to tensor
y = torch.tensor(cohort_new_indexed.select("EPI").toPandas().values.astype(int), dtype=torch.long).squeeze()

# Ensure labels are aligned with IDs in x_person
# (Assuming x_person is already sorted by person ID index)
# If not, you might need to sort both x_person and y by ID index





In [ ]:
from torch_geometric.data import HeteroData, DataLoader
# Create HeteroData object
data1 = HeteroData()
data1.x = x_combined.to(torch.int64)
data1.edge_index = edge_index_person_to_diagnosis
data1.edge_attr = edge_attr
data1.y = y


In [ ]:
data1 # 6661834

HeteroData(
  x=[1701705, 4],
  edge_index=[2, 6661834],
  edge_attr=[6661834, 1],
  y=[1701685]
)

In [ ]:
data1.x

tensor([[      0,       0,       0,       0],
        [      1,       0,       0,       0],
        [      2,       0,       0,       0],
        ...,
        [1701702,      33,       2,       1],
        [1701703,      80,       2,       1],
        [1701704,       0,       3,       1]])

In [ ]:
# prompt: write data1 into h5

import h5py

# Create an h5 file
with h5py.File('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag20_first.h5', 'w') as f:
  # Store node features
  f.create_dataset('x', data=data1.x.numpy())

  # Store edge index
  f.create_dataset('edge_index', data=data1.edge_index.numpy())

  # Store edge attributes
  f.create_dataset('edge_attr', data=data1.edge_attr.numpy())

  # Store labels
  f.create_dataset('y', data=data1.y.numpy())


# Run from the following if using existing h5 data

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Check if GPU is available
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# prompt: read from the h5 file into data1
import h5py

from torch_geometric.data import HeteroData
data1 = HeteroData()
with h5py.File('/content/drive/MyDrive/UCO/projects/epilepsy/EPI_heterodata_diag20_first.h5', 'r') as f:
  x = torch.tensor(f['x'][:])
  edge_index = torch.tensor(f['edge_index'][:])
  edge_attr = torch.tensor(f['edge_attr'][:])
  y = torch.tensor(f['y'][:])

# Update the HeteroData object
data1.x = x[:,1:]
data1.edge_index = edge_index
data1.edge_attr = edge_attr
data1.y = y


In [ ]:
data1

HeteroData(
  x=[1701705, 3],
  edge_index=[2, 6661834],
  edge_attr=[6661834, 1],
  y=[1701685]
)

In [ ]:
# prompt: aggregate the edges using the highest edge attr

from torch_geometric.utils import to_undirected
import builtins

# Convert the graph to undirected if needed
# edge_index_undirected = to_undirected(data1.edge_index)
edge_index_undirected = data1.edge_index

# Get a list of unique edges with their attributes
unique_edges = []
for i in range(edge_index_undirected.shape[1]):
  u, v = edge_index_undirected[0, i].item(), edge_index_undirected[1, i].item()
  attr = data1.edge_attr[i].item()
  unique_edges.append(((min(u, v), builtins.max(u, v)), attr))

# Aggregate edges based on the highest edge attribute
aggregated_edges = {}
for (u, v), attr in unique_edges:
  if (u, v) not in aggregated_edges or attr > aggregated_edges[(u, v)]:
    aggregated_edges[(u, v)] = attr

# Create a new edge index and edge attribute tensor
new_edge_index = torch.tensor(list(aggregated_edges.keys())).t().contiguous()
new_edge_attr = torch.tensor(list(aggregated_edges.values()), dtype=torch.float).unsqueeze(1)

# Update the data object
data1.edge_index = new_edge_index
data1.edge_attr = new_edge_attr

print(data1)


HeteroData(
  x=[1701705, 3],
  edge_index=[2, 2538111],
  edge_attr=[2538111, 1],
  y=[1701685]
)


In [ ]:
data1.edge_index

tensor([[     18,      14,      19,  ...,      16,      11,       6],
        [1484397, 1484397, 1484397,  ...,  138541,  138541,  138541]])

In [ ]:
# prompt: create a spark dataframe based on data1.edge_index

# Convert the edge_index tensor to a list of tuples
edge_list = [(data1.edge_index[0, i].item(), data1.edge_index[1, i].item()) for i in range(data1.edge_index.shape[1])]


NameError: name 'spark' is not defined

In [ ]:

# Create a PySpark DataFrame from the edge list
edge_df = spark.createDataFrame(edge_list, ['src', 'dst'])

# Show the resulting DataFrame
edge_df.show()


+---+---+
|src|dst|
+---+---+
|  0| 24|
|  0| 25|
|  0| 44|
|  0| 61|
|  0| 84|
|  0| 94|
|  0|141|
|  0|146|
|  0|147|
|  0|156|
|  0|157|
|  0|177|
|  0|182|
|  0|186|
|  0|235|
|  0|244|
|  0|253|
|  0|267|
|  0|273|
|  0|274|
+---+---+
only showing top 20 rows



In [ ]:
edge_df.count()

2538111

In [ ]:
edge_df.distinct().count()

2538111

In [ ]:
# prompt: get the max value of edge df in each column

from pyspark.sql.functions import max

# Find the maximum value for each column in edge_df
max_values = edge_df.select([max(c) for c in edge_df.columns])

# Show the maximum values
max_values.show()


+--------+--------+
|max(src)|max(dst)|
+--------+--------+
|      19| 1701703|
+--------+--------+



In [ ]:
# prompt: make data1 heterodata by defining x_dict instead of x. Use diagnosis for the 0-19 nodes. The rest of the nodes are person

data2 = HeteroData()

# Create a dictionary for node features
data2['diagnosis'].x = data1.x[:20][:,0].unsqueeze(1).float().to(device)
data2['person'].x = data1.x[20:].float().to(device)

# Store edge index
data2['diagnosis','to','person'].edge_index = data1.edge_index.to(device)
# data1['diagnosis', 'is_associated_with', 'person'].edge_index = edge_index_diagnosis_to_person
data2['diagnosis','to','person'].edge_attr = data1.edge_attr.to(device)

data2.y = data1.y.to(device)

print(data2['diagnosis','to','person'].edge_index)


tensor([[     18,      14,      19,  ...,      16,      11,       6],
        [1484377, 1484377, 1484377,  ...,  138521,  138521,  138521]])


In [ ]:
# prompt: subtract 20 for all the index in the person indexes within data2['person','to','diagnosis'].edge_index

import torch

# Assuming data2['person','to','diagnosis'].edge_index is a tensor
edge_index = data2['diagnosis','to','person'].edge_index

# Find the indices where the node type is 'person'
person_indices = edge_index[1] >= 20


# Subtract 20 from the person indices within edge_index[1]
edge_index[1][person_indices] -= 20

# Update the edge index in data2
data2['diagnosis','to','person'].edge_index = edge_index.to(device)

print(data2['diagnosis','to','person'].edge_index)


tensor([[     18,      14,      19,  ...,      16,      11,       6],
        [1484357, 1484357, 1484357,  ...,  138501,  138501,  138501]])


In [ ]:
# prompt: switch the order of the first and second row of edge_index

edge_index1 = data2['diagnosis','to','person'].edge_index.clone()
edge_index1[1] = edge_index[0]
edge_index1[0] = edge_index[1]
data2['person','to','diagnosis'].edge_index = edge_index1.to(device)


In [ ]:
print(data2['person','to','diagnosis'].edge_index)

tensor([[1484357, 1484357, 1484357,  ...,  138501,  138501,  138501],
        [     18,      14,      19,  ...,      16,      11,       6]])


In [ ]:
# data2['diagnosis','to','person'].edge_index = edge_index.transpose(0,1)
print(data2['diagnosis','to','person'].edge_index)

tensor([[     18,      14,      19,  ...,      16,      11,       6],
        [1484357, 1484357, 1484357,  ...,  138501,  138501,  138501]])


In [ ]:
# # prompt: add normalization layer

# # prompt: modify the code to use GPU to run

# import torch
# from torch_geometric.data import HeteroData
# import torch.nn.functional as F
# from torch_geometric.nn import GATConv
# from torch_geometric.data import HeteroData, DataLoader
# from torch_geometric.nn import SAGEConv, to_hetero

# # Check if GPU is available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Using device: {device}")

# # Define the GAT model
# class GAT(torch.nn.Module):
#     def __init__(self, input_features, edge_features, hidden_channels, convhidden_channels, out_channels, num_heads):
#         super(GAT, self).__init__()
#         self.gat1 = GATConv(input_features, hidden_channels, heads=num_heads, concat=True, dropout = 0.6).to(device)
#         self.gat2 = GATConv(hidden_channels * num_heads, convhidden_channels, heads=1, concat=False,dropout = 0.6).to(device)
#         self.edge_mlp = torch.nn.Sequential(
#             torch.nn.Linear(edge_features, hidden_channels),
#             torch.nn.ReLU(),
#             torch.nn.Linear(hidden_channels, hidden_channels)
#         ).to(device)
#         self.conv1 = torch.nn.Conv1d(in_channels=convhidden_channels, out_channels=out_channels, kernel_size=3, padding=1)
#         self.mlp = torch.nn.Linear(convhidden_channels, out_channels)
#         self.layer_norm = torch.nn.LayerNorm(convhidden_channels) # Add normalization layer

#     def forward(self, x, edge_index, edge_attr):
#         x = x.to(device)
#         edge_index = edge_index.to(device)
#         edge_attr = edge_attr.to(device)
#         edge_attr_transformed = self.edge_mlp(edge_attr)
#         # x = F.dropout(x, p=0.6)
#         x = self.gat1(x, edge_index, edge_attr=edge_attr_transformed)
#         x = F.elu(x)
#         # x = F.dropout(x, p=0.6)
#         x = self.gat2(x, edge_index, edge_attr=edge_attr_transformed)
#         x = F.elu(x)
#         # x = self.layer_norm(x) # Apply normalization
#         # x = torch.permute(x, (0,2,1))
#         # x = self.conv1(x)
#         x = self.mlp(x)
#         # return x
#         return F.log_softmax(x, dim=1)

# # ... (rest of the code remains the same)

# # Move data to GPU
# data1.x = data1.x.to(torch.float32).to(device)
# data1.edge_index = data1.edge_index.to(device)
# data1.edge_attr = data1.edge_attr.to(torch.float32).to(device)
# data1.y = data1.y.to(device)

# # Instantiate the model and move it to GPU
# model = GAT(input_features=3, edge_features=1, hidden_channels=8, convhidden_channels=32, out_channels=2, num_heads=2).to(device)


Using device: cuda


In [ ]:
print(data2.metadata())
print(data2.x_dict)
print(data2.edge_index_dict)

(['diagnosis', 'person'], [('person', 'to', 'diagnosis'), ('diagnosis', 'to', 'person')])
{'diagnosis': tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]), 'person': tensor([[46.,  2.,  1.],
        [ 6.,  1.,  1.],
        [31.,  2.,  1.],
        ...,
        [33.,  2.,  1.],
        [80.,  2.,  1.],
        [ 0.,  3.,  1.]])}
{('person', 'to', 'diagnosis'): tensor([[1484377, 1484377, 1484377,  ...,  138521,  138521,  138521],
        [     18,      14,      19,  ...,      16,      11,       6]]), ('diagnosis', 'to', 'person'): tensor([[     18,      14,      19,  ...,      16,      11,       6],
        [1484377, 1484377, 1484377,  ...,  138521,  138521,  138521]])}


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, HeteroConv
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, to_hetero,Linear

class HeteroGAT2(torch.nn.Module):
    def __init__(self, input_features, edge_features, hidden_channels, convhidden_channels, out_channels, num_heads, metadata):
        super().__init__()

        self.gat1 = HeteroConv({
            ('person', 'to', 'diagnosis'): GATConv((-1, -1), hidden_channels, heads=num_heads, concat=True,  add_self_loops=False),
            # ('person', 'to', 'diagnosis'): GATConv((-1, -1), hidden_channels
            ('diagnosis', 'to', 'person'): GATConv((-1, -1), hidden_channels, heads=num_heads, concat=True,  add_self_loops=False),
        })
        self.gat2 = HeteroConv({
            ('person', 'to', 'diagnosis'): GATConv(hidden_channels * num_heads, convhidden_channels, heads=1, concat=False, add_self_loops=False),
            # ('person', 'to', 'diagnosis'): GATConv((-1, -1), hidden_channels
            ('diagnosis', 'to', 'person'): GATConv(hidden_channels * num_heads, convhidden_channels, heads=1, concat=False, add_self_loops=False),
        })


        self.lin = torch.nn.Linear(convhidden_channels, out_channels)
        self.edge_mlp = torch.nn.Sequential(
          torch.nn.Linear(edge_features, hidden_channels),
          torch.nn.ReLU(),
          torch.nn.Linear(hidden_channels, hidden_channels)
        )
        # self.conv1 = torch.nn.Conv1d(in_channels=convhidden_channels, out_channels=out_channels, kernel_size=3, padding=1)
        # self.mlp = torch.nn.Linear(convhidden_channels, out_channels)
        self.layer_norm = torch.nn.LayerNorm(convhidden_channels) # Add normalization layer

    def forward(self, x_dict, edge_index_dict, edge_attr_dict):
    #     print(x_dict)
        # print(edge_attr_dict)


        edge_attr_transformed = self.edge_mlp(edge_attr_dict[('diagnosis', 'to', 'person')])
        x_dict = self.gat1(x_dict, edge_index_dict, edge_attr_dict)
        x_dict = {key: x.relu() for key, x in x_dict.items()}

        # x_dict = F.elu(x_dict)
        # x = F.dropout(x, p=0.6)
        x_dict = self.gat2(x_dict, edge_index_dict, edge_attr_dict)
        x_dict = {key: x.relu() for key, x in x_dict.items()}
        # x_dict = F.elu(x_dict)
        # x = self.layer_norm(x) # Apply normalization
        # x = torch.permute(x, (0,2,1))
        # x = self.conv1(x)
        x = x_dict['person']
        # x = self.mlp(x)

        # print(x_dict)

        # x = F.relu(x)
        x = self.lin(x)
        # x = F.relu(x)



        # return F.log_softmax(x, dim=1)
        return x


# Instantiate the model and move it to GPU
model = HeteroGAT2(input_features=3, edge_features=1, hidden_channels=16, convhidden_channels=16, out_channels=1, num_heads=2, metadata=data2.metadata()).to(device)  # 2 output channels for binary classification
# model = to_hetero(model, data2.metadata(), aggr='sum')

In [ ]:
data2

HeteroData(
  y=[1701685],
  diagnosis={ x=[20, 1] },
  person={ x=[1701685, 3] },
  (diagnosis, to, person)={
    edge_index=[2, 2538111],
    edge_attr=[2538111, 1],
  },
  (person, to, diagnosis)={ edge_index=[2, 2538111] }
)

In [ ]:
data_balanced2

HeteroData(
  x=[205394, 3],
  edge_index=[2, 279241],
  edge_attr=[279241, 1],
  y=[205374],
  person={ x=[205374, 3] },
  diagnosis={ x=[20, 1] },
  (person, to, diagnosis)={
    edge_attr=[279241, 1],
    edge_index=[2, 279241],
  },
  (diagnosis, to, person)={ edge_index=[2, 279241] }
)

In [ ]:
data_balanced2.edge_index

tensor([[     0,      0,      0,  ...,     19,     19,     19],
        [123086, 116937,     36,  ..., 187119, 102692, 176833]])

In [ ]:
from torch_geometric.data import HeteroData, DataLoader
loader = DataLoader([data2], batch_size=10, shuffle=True)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# prompt: train the GAT model
from torch_geometric.utils import subgraph
# Create a DataLoader
# loader = DataLoader([data1], batch_size=10, shuffle=True)


# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.BCELoss()
import torch
from tqdm import tqdm

# ... (rest of your code)

# Training loop
model.train()
for epoch in range(50):  # Number of epochs
    total_loss = 0
    progress_bar = tqdm(loader, desc=f"Epoch {epoch + 1}/{100}")
    # print(batch)
    for batch in progress_bar:
        optimizer.zero_grad()
        out = model(batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict).to(device)
        out = F.sigmoid(out)
        loss = criterion(out.squeeze(1).float(), batch.y.float())
        # print(out)
        # loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})  # Display loss for each batch

    average_loss = total_loss / len(loader)
    print(f'Epoch {epoch + 1}, Average Loss: {average_loss}')


Epoch 1/100: 100%|██████████| 1/1 [00:04<00:00,  4.02s/it, loss=0.508]


Epoch 1, Average Loss: 0.5084675550460815


Epoch 2/100: 100%|██████████| 1/1 [00:03<00:00,  3.50s/it, loss=0.499]


Epoch 2, Average Loss: 0.499028742313385


Epoch 3/100: 100%|██████████| 1/1 [00:03<00:00,  3.36s/it, loss=0.492]


...

Epoch 48, Average Loss: 0.4548955261707306


Epoch 49/100: 100%|██████████| 1/1 [00:03<00:00,  3.92s/it, loss=0.455]


Epoch 49, Average Loss: 0.454674631357193


Epoch 50/100:   0%|          | 0/1 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
from torch_geometric.utils import subgraph
def dataloader_subgraph(data, batch_size, preserve):
  # preserve is the number of samples at first to retain in each batch
  datalist = []
  num_batches = (data.x.shape[0] - preserve) // batch_size
  print('num_batches',num_batches)
  for i in range(0,num_batches):
    indices = torch.cat([torch.arange(0,20).to(device),torch.arange(preserve + i * batch_size, preserve + (i + 1) * batch_size, dtype=int).to(device)])
    subgraph_edge_index, subgraph_edge_attr = subgraph(indices, data.edge_index, data.edge_attr, relabel_nodes= True)
    # print('ran1')
    subgraphed = HeteroData()
    subgraphed.x = data.x[indices]
    subgraphed.edge_index = subgraph_edge_index
    subgraphed.edge_attr = subgraph_edge_attr
    subgraphed.y = data.y[preserve + i * batch_size:preserve + (i + 1) * batch_size]
    datalist.append(subgraphed)

  # indices = torch.cat([torch.arange(0,20).to(device),torch.range(preserve + (num_batches - 1) * batch_size, data.x.shape[0], dtype = int).to(device)])
  # subgraph_edge_index, subgraph_edge_attr = subgraph(indices, data.edge_index, data.edge_attr)
  # # print(subgraph_edge_index, subgraph_edge_attr)
  # # print('ran2')
  # subgraphed = HeteroData()
  # subgraphed.x = data.x[indices]
  # subgraphed.edge_index = subgraph_edge_index
  # datalist.append(subgraphed)

  return datalist


In [ ]:
from torch_geometric.data import HeteroData, DataLoader
loader = dataloader_subgraph(data_balanced2, batch_size=10)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

TypeError: dataloader_subgraph() missing 1 required positional argument: 'preserve'

In [ ]:
# prompt: add progress bar and display loss. for each batch write loss

import torch
from tqdm import tqdm

# ... (rest of your code)

# Training loop
model.train()
for epoch in range(100):  # Number of epochs
    total_loss = 0
    progress_bar = tqdm(loader, desc=f"Epoch {epoch + 1}/{100}")
    for batch in progress_bar:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.edge_attr).to(device)
        loss = criterion(out[20:], batch.y)  # Only consider the persons for the loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})  # Display loss for each batch

    average_loss = total_loss / len(loader)
    print(f'Epoch {epoch + 1}, Average Loss: {average_loss}')


Epoch 1/100: 100%|██████████| 1701/1701 [00:12<00:00, 139.04it/s, loss=0.249]


Epoch 1, Average Loss: 0.22797779763431705


Epoch 2/100: 100%|██████████| 1701/1701 [00:12<00:00, 137.70it/s, loss=0.248]


Epoch 2, Average Loss: 0.22797430193227997


Epoch 3/100: 100%|██████████| 1701/1701 [00:12<00:00, 136.90it/s, loss=0.249]


...

Epoch 4, Average Loss: 0.22796296796540805


Epoch 5/100: 100%|██████████| 1701/1701 [00:12<00:00, 138.98it/s, loss=0.249]


Epoch 5, Average Loss: 0.22795535255221322


Epoch 6/100:  43%|████▎     | 724/1701 [00:05<00:06, 139.97it/s, loss=0.211]


KeyboardInterrupt: 

In [ ]:
# prompt: get a confusion matrix for the trained model

# Evaluate the model
model.eval()
with torch.no_grad():
  out = model(data2.x_dict, data2.edge_index_dict, data2.edge_attr_dict)
  _, predicted = torch.max(out, dim=1)  # Predictions for persons only
  # predicted = out > 0.5  # Predictions for persons only

# Get the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(data2.y.cpu(), predicted.cpu())
print(cm)


[[1598998       0]
 [ 102687       0]]


In [ ]:
# prompt: get a classification report

from sklearn.metrics import classification_report
print(classification_report(data_balanced2.y.cpu(), predicted.cpu()))


ValueError: Found input variables with inconsistent numbers of samples: [205374, 1701685]

In [ ]:
from torch_geometric.utils import subgraph

# Count the number of samples in each class
class_counts = torch.bincount(data1.y)
print('class counts = ',class_counts)

# Find the minority class
minority_class = torch.argmin(class_counts)
majority_class = torch.argmax(class_counts)

print('minority_class: ',minority_class)
print('majority_class: ',majority_class)

# Calculate the number of samples to keep from the majority class
num_samples_to_keep = class_counts[minority_class]

# Get indices of majority class samples
majority_indices = (data1.y == majority_class).nonzero().squeeze() + 20
print('majority_indices: ',majority_indices)
print('majority_indices_size: ',majority_indices.shape)

# Randomly select majority class samples to keep
random_indices = torch.randperm(majority_indices.size(0))[:num_samples_to_keep]
selected_majority_indices = majority_indices[random_indices]

print('selected_majority_indices: ',selected_majority_indices)

# Combine indices of minority and selected majority samples
balanced_indices = torch.cat([(data1.y == minority_class).nonzero().squeeze() + 20, selected_majority_indices])
print('balanced_indices:',balanced_indices)
# Subset the data using the balanced indices
x_balanced = data1.x[balanced_indices]

padding = torch.zeros([20,x_balanced.shape[1]])
x_balanced_padded = torch.cat([padding, x_balanced.cpu() ], dim=0).float()
y_balanced = data1.y[balanced_indices - 20]
# y_balanced = torch.cat([torch.full([20],2).to(device),data1.y[balanced_indices - 20]],dim=0)


print(x_balanced)
print(x_balanced_padded)
print(y_balanced)

class counts =  tensor([1598998,  102687])
minority_class:  tensor(1)
majority_class:  tensor(0)
majority_indices:  tensor([     21,      22,      23,  ..., 1701702, 1701703, 1701704])
majority_indices_size:  torch.Size([1598998])
selected_majority_indices:  tensor([1460899,   86855,  938915,  ..., 1022942,   77170,  716832])
balanced_indices: tensor([     20,      55,      62,  ..., 1022942,   77170,  716832])
tensor([[46,  2,  1],
        [81,  2,  1],
        [29,  2,  1],
        ...,
        [ 2,  1,  1],
        [75,  2,  6],
        [ 7,  2,  1]])
tensor([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ...,
        [ 2.,  1.,  1.],
        [75.,  2.,  6.],
        [ 7.,  2.,  1.]])
tensor([1, 1, 1,  ..., 0, 0, 0])


In [ ]:
import torch
from torch_geometric.data import HeteroData
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import HeteroData, DataLoader

import numpy as np

# extended_indices = torch.cat([balanced_indices.to(device),data1.x[:,0][personid_cutoff:]],axis=0).int() # I have to attach the diagnosis ids because the diagnosis ids are not appearing in the balanced indices, which will return empty in subgraph
extended_indices = torch.cat([torch.tensor(range(0,20)).to(device), balanced_indices.to(device)],axis=0).int() # I have to attach the diagnosis ids because the diagnosis ids are not appearing in the balanced indices, which will return empty in subgraph
# print(extended_indices)
edge_index_balanced, edge_attr_balanced = subgraph(extended_indices, data2.edge_index.to(device), data2.edge_attr.to(device),relabel_nodes=True)
# edge_index_balanced, edge_attr_balanced = subgraph(extended_indices, data1.edge_index, data1.edge_attr)
print(edge_index_balanced)

# x_balanced[:,0] = torch.cat([torch.from_numpy(np.arange(0,x_balanced.shape[0])),data1.x[:,0][personid_cutoff:]],axis=0)
# print(x_balanced)

# Create a new HeteroData object with balanced data
data_balanced = HeteroData()
data_balanced.x = x_balanced_padded.to(device)
data_balanced.edge_index = edge_index_balanced.to(device)
data_balanced.edge_attr = edge_attr_balanced.to(device)
data_balanced.y = y_balanced.to(device)

print(data_balanced.x.shape)
print(data_balanced.y.shape)


AttributeError: 'HeteroData' has no attribute 'edge_index'

In [ ]:
data_balanced

HeteroData(
  x=[205394, 3],
  edge_index=[2, 817370],
  edge_attr=[817370, 1],
  y=[205374]
)

In [ ]:
data_balanced2 = data_balanced.clone()
data_balanced2['person'].x = data_balanced.x[20:]
data_balanced2['diagnosis'].x = data_balanced.x[:20][:,0].unsqueeze(1)
data_balanced2[('person', 'to', 'diagnosis')].edge_attr = data_balanced.edge_attr.to(device)
edge_index = data_balanced.edge_index
person_indices = edge_index[0] >= 20
edge_index[0][person_indices] -= 20

# Update the edge index in data2
data_balanced2['person','to','diagnosis'].edge_index = edge_index.to(device)
data_balanced2['person','to','diagnosis'].edge_index = data_balanced.edge_index.to(device)

edge_index1 = edge_index.clone()
edge_index1[1] = edge_index[0]
edge_index1[0] = edge_index[1]
data_balanced2['diagnosis','to','person'].edge_index = edge_index1.to(device)

In [ ]:
data_balanced2

HeteroData(
  x=[205394, 3],
  edge_index=[2, 817370],
  edge_attr=[817370, 1],
  y=[205374],
  person=[205374, 3],
  diagnosis=[20, 1],
  (person, to, diagnosis)={
    edge_attr=[817370, 1],
    edge_index=[2, 817370],
  },
  (diagnosis, to, person)={ edge_index=[2, 817370] }
)

In [ ]:
# prompt: make a train mask with random sampling

# Generate a random permutation of indices
random_indices = torch.randperm(len(y_balanced))+20
train_ratio = 0.8
# Calculate the number of training samples
num_train = int(train_ratio * len(y_balanced))

# Create the train mask based on the random permutation
train_mask = torch.zeros(len(y_balanced)+20, dtype=torch.bool)
train_mask[random_indices[:num_train]] = True

# Create the test mask (complement of the train mask)
test_mask = ~train_mask
test_mask[:20] = False


In [ ]:
# prompt: use subgraph to get train data and test data from data_balanced


# Get indices of training and testing nodes
train_indices = torch.where(train_mask)[0]
test_indices = torch.where(test_mask)[0]

# Use subgraph to extract training and testing data
train_data = data_balanced.clone()
train_data.edge_index, train_data.edge_attr = subgraph(
    torch.cat([data1.x[:,0][:20],train_indices.to(device)],axis=0).int(), data_balanced.edge_index, data_balanced.edge_attr, relabel_nodes=True)
train_data.y = data_balanced.y[train_mask[20:]]
train_data.x = train_data.x[train_indices].to(device).float()

test_data = data_balanced.clone()
test_data.edge_index, test_data.edge_attr = subgraph(
    torch.cat([data1.x[:,0][:20], test_indices.to(device)],axis=0).int(), data_balanced.edge_index, data_balanced.edge_attr, relabel_nodes=True)
test_data.y = data_balanced.y[test_mask[20:]]
test_data.x = test_data.x[test_indices].to(device).float()

# Create DataLoaders for training and testing
train_loader = DataLoader([train_data], batch_size=10, shuffle=True)
test_loader = DataLoader([test_data], batch_size=10, shuffle=False)


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
test_data.x

tensor([[2.0000e+01, 4.6956e+01, 2.0000e+00, 1.0000e+00],
        [1.3200e+02, 8.3688e+01, 1.0000e+00, 1.0000e+00],
        [1.7300e+02, 0.0000e+00, 1.0000e+00, 1.0000e+00],
        ...,
        [5.6938e+05, 2.0525e+00, 1.0000e+00, 1.0000e+00],
        [1.6360e+06, 8.1744e+01, 2.0000e+00, 1.0000e+00],
        [1.3943e+06, 6.8331e+01, 1.0000e+00, 1.0000e+00]])

In [ ]:
test_data.y

tensor([1, 1, 1,  ..., 0, 0, 0])

In [ ]:
test_data.edge_index

tensor([[20779, 20779, 20779,  ..., 27561, 27561, 27561],
        [    6,     7,    16,  ...,     9,     9,     0]])

In [ ]:
train_data.edge_index

tensor([[138537,  83217,  48724,  ..., 110910, 110910, 110910],
        [     7,      9,      7,  ...,     16,     11,      6]])

In [ ]:
train_indices

tensor([    21,     22,     23,  ..., 205387, 205388, 205390])

In [ ]:
test_indices

tensor([    20,     27,     29,  ..., 205391, 205392, 205393])

In [ ]:
# prompt: count the number of y=0 and y=1 in train and test data

train_y_counts = torch.bincount(train_data.y.cpu())
test_y_counts = torch.bincount(test_data.y.cpu())

print("Training data y counts:")
print(f"y=0: {train_y_counts[0]}")
print(f"y=1: {train_y_counts[1]}")

print("\nTesting data y counts:")
print(f"y=0: {test_y_counts[0]}")
print(f"y=1: {test_y_counts[1]}")


Training data y counts:
y=0: 82102
y=1: 82197

Testing data y counts:
y=0: 20585
y=1: 20490


In [ ]:
# prompt: train the GAT model
# import torch_geometric.transforms as T
# transform = T.Compose([
#     T.RandomNodeSplit(num_val=0.1, num_test=0.2),
# ])
# Create a DataLoader
# loader = DataLoader([data_balanced], batch_size=100, shuffle=True)
loader = dataloader_subgraph(data_balanced, 30000, 20)
# print(loader)

# # Instantiate the model
model2= GAT(input_features=3,
                  edge_features=1, hidden_channels=32, convhidden_channels=32, out_channels=2, num_heads=50).to(device)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model2.parameters(), lr=0.05)
# criterion = torch.nn.BCEWithLogitsLoss()
criterion = torch.nn.CrossEntropyLoss()
# for batch in loader:
#   print(batch.x)
#   print(batch.edge_index)
#   print(batch.edge_attr.shape)
#   print(batch.y)
#   break


# # Training loop

# model2.train()
# for epoch in range(100):  # Number of epochs
#     for batch in loader:
#         optimizer.zero_grad()
#         # print(batch.x)
#         out = model2(batch.x, batch.edge_index, batch.edge_attr).to(device)
#         # out = F.sigmoid(out)
#         # out = F.log_softmax(out, dim=1)
#         # out = out.squeeze().to(torch.float32)
#         # print(out)
#         # print(batch.y.shape)
#         # print(out[:x_person.shape[0]-1])
#         # loss = criterion(out[20:], batch.y.to(torch.float32))  # Only consider the persons for the loss
#         loss = criterion(out[20:], batch.y)  # Only consider the persons for the loss
#         loss.backward()
#         optimizer.step()

#     if epoch % 10 == 0:  # Print loss every 10 epochs
#         print(f'Epoch {epoch}, Loss: {loss.item()}')
import torch
from tqdm import tqdm

# ... (rest of your code)

# Training loop
model2.train()
for epoch in range(100):  # Number of epochs
    total_loss = 0
    progress_bar = tqdm(loader, desc=f"Epoch {epoch + 1}/{100}")
    for batch in progress_bar:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.edge_attr).to(device)
        loss = criterion(out[20:], batch.y)  # Only consider the persons for the loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})  # Display loss for each batch

    average_loss = total_loss / len(loader)
    print(f'Epoch {epoch + 1}, Average Loss: {average_loss}')


num_batches 6


Epoch 1/100: 100%|██████████| 6/6 [00:00<00:00, 74.58it/s, loss=0.0637]


Epoch 1, Average Loss: 1.6192397996783257


Epoch 2/100: 100%|██████████| 6/6 [00:00<00:00, 79.99it/s, loss=0.0637]


Epoch 2, Average Loss: 1.619463545580705


...

Epoch 98, Average Loss: 1.6196175180375576


Epoch 99/100: 100%|██████████| 6/6 [00:00<00:00, 111.13it/s, loss=0.0637]


Epoch 99, Average Loss: 1.619420163333416


Epoch 100/100: 100%|██████████| 6/6 [00:00<00:00, 114.36it/s, loss=0.0637]

Epoch 100, Average Loss: 1.6192755637069542


In [ ]:
# prompt: evaluate on test_loader using confusion matrix

# Instantiate the model
# model3= GAT(input_features=3,
#                   edge_features=1, hidden_channels=32, convhidden_channels= 32, out_channels=2, num_heads=20).to(device)

# Evaluate the model on the test set
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for batch in loader:
        print(batch)
        out = model(batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict)
        # out = F.sigmoid(out)
        print(out)
        # predicted = (out[20:] > 0.5).float()
        _, predicted = torch.max(out, dim=1)
        y_true.extend(batch.y.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())


# Calculate and print the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)


HeteroDataBatch(
  x=[205394, 3],
  edge_index=[2, 817370],
  edge_attr=[817370, 1],
  y=[205374],
  person={
    x=[205374, 3],
    batch=[205374],
    ptr=[2],
  },
  diagnosis={
    x=[20, 1],
    batch=[20],
    ptr=[2],
  },
  (person, to, diagnosis)={
    edge_attr=[817370, 1],
    edge_index=[2, 817370],
  },
  (diagnosis, to, person)={ edge_index=[2, 817370] }
)
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]], device='cuda:0')
Confusion Matrix:
[[102687      0]
 [102687      0]]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67    102687
           1       0.00      0.00      0.00    102687

    accuracy                           0.50    205374
   macro avg       0.25      0.50      0.33    205374
weighted avg       0.25      0.50      0.33    205374



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# prompt: get AUC

from sklearn.metrics import roc_auc_score

# Assuming y_true and y_pred are lists of true and predicted labels
# If you have probabilities instead of predicted labels, you need to use those

# Calculate AUC
try:
  auc = roc_auc_score(y_true, y_pred)
  print(f"AUC: {auc}")
except ValueError:
  print("AUC could not be calculated.")



AUC: 0.4790441643365922
